In [1]:
import matplotlib.pyplot as plt
import math
import pandas_datareader as web
import pandas as pd
import numpy as np
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
plt.style.use('fivethirtyeight')

%matplotlib inline

Using TensorFlow backend.


In [7]:
df = pd.read_excel('./data/Customer_attrition.xlsx', header=3)

In [12]:
df = df.iloc[:,1:]

In [13]:
df.head()

,Customer Name,Site Number,Industry,Sub ISU Account Designation,Program Type,Site Anniversary Date,Market,SS Rep,SS FLM,Software Name,...,Attrition Type,Dropped SS since Q1 2020,Current S&S Quantity,SS End Date,Latest Renewal or License Part number,Reinstatement Part Number,PA Renewal Contact First Name,PA Renewal Contact Last Name,Contact Phone Number,Email
0,SONEPAR MANAGEMENT US INC,3002962,Enterprise,Wholesale- Durables,Passport Advantage,04/01,Distribution Market,"Gutierrez, Markus P.","Muncrief, Jennifer M.",Cognos Analytic Server for Non-Production Proc...,...,Full Drop to 0,210,0,2020-03-31,E064KLL,D076FLL,Angela,Alston‑Roberts,2675072738,angela.alston-roberts@sonepar-us.com
1,SONEPAR MANAGEMENT US INC,3002962,Enterprise,Wholesale- Durables,Passport Advantage,04/01,Distribution Market,"Gutierrez, Markus P.","Muncrief, Jennifer M.",Cognos Analytic Server Processor Value Unit,...,Full Drop to 0,420,0,2020-03-31,E08BCLL,D0C13LL,Angela,Alston‑Roberts,2675072738,angela.alston-roberts@sonepar-us.com
2,SONEPAR MANAGEMENT US INC,3002962,Enterprise,Wholesale- Durables,Passport Advantage,04/01,Distribution Market,"Gutierrez, Markus P.","Muncrief, Jennifer M.",Cognos Business Intelligence Analytics Adminis...,...,Full Drop to 0,7,0,2020-03-31,E0K30LL,D17BBLL,Angela,Alston‑Roberts,2675072738,angela.alston-roberts@sonepar-us.com
3,SONEPAR MANAGEMENT US INC,3002962,Enterprise,Wholesale- Durables,Passport Advantage,04/01,Distribution Market,"Gutierrez, Markus P.","Muncrief, Jennifer M.",Cognos Business Intelligence Analytics Explore...,...,Full Drop to 0,20,0,2020-03-31,E0K2DLL,D175ELL,Angela,Alston‑Roberts,2675072738,angela.alston-roberts@sonepar-us.com
4,SONEPAR MANAGEMENT US INC,3002962,Enterprise,Wholesale- Durables,Passport Advantage,04/01,Distribution Market,"Gutierrez, Markus P.","Muncrief, Jennifer M.",Cognos Business Intelligence Analytics Explore...,...,Full Drop to 0,400,0,2020-03-31,E0K2FLL,D175ILL,Angela,Alston‑Roberts,2675072738,angela.alston-roberts@sonepar-us.com


In [18]:
df_placeholder = []
for i in df.columns:
    i = i.replace(' ', '_')
    df_placeholder.append(i)
    
df_placeholder

['Customer_Name',
 'Site_Number',
 'Industry',
 'Sub_ISU_Account_Designation',
 'Program_Type',
 'Site_Anniversary_Date',
 'Market',
 'SS_Rep',
 'SS_FLM',
 'Software_Name',
 'Brand',
 'Reseller_Name',
 'Payer_Name',
 'USD_Extended_Price',
 'Attrition_Type',
 'Dropped_SS_since_Q1_2020',
 'Current_S&S_Quantity',
 'SS_End_Date',
 'Latest_Renewal_or_License_Part_number',
 'Reinstatement_Part_Number',
 'PA_Renewal_Contact_First_Name',
 'PA_Renewal_Contact_Last_Name',
 'Contact_Phone_Number',
 'Email']

In [19]:
df.columns = df_placeholder
df.columns

Index(['Customer_Name', 'Site_Number', 'Industry',
       'Sub_ISU_Account_Designation', 'Program_Type', 'Site_Anniversary_Date',
       'Market', 'SS_Rep', 'SS_FLM', 'Software_Name', 'Brand', 'Reseller_Name',
       'Payer_Name', 'USD_Extended_Price', 'Attrition_Type',
       'Dropped_SS_since_Q1_2020', 'Current_S&S_Quantity', 'SS_End_Date',
       'Latest_Renewal_or_License_Part_number', 'Reinstatement_Part_Number',
       'PA_Renewal_Contact_First_Name', 'PA_Renewal_Contact_Last_Name',
       'Contact_Phone_Number', 'Email'],
      dtype='object')

In [55]:
cust_df = df.groupby('Customer_Name').sum().sort_values('Dropped_SS_since_Q1_2020',ascending=False).reset_index()[['Customer_Name','Dropped_SS_since_Q1_2020','Current_S&S_Quantity']]

In [57]:
cust_df['ss_difference'] = cust_df['Dropped_SS_since_Q1_2020'] - cust_df['Current_S&S_Quantity'] 

In [69]:
cust_df.query('ss_difference < 0').sort_values('ss_difference').head(10)

,Customer_Name,Dropped_SS_since_Q1_2020,Current_S&S_Quantity,ss_difference
31,"Prime Therapeutics, LLC",3424,18103,-14679
39,Harland Clarke Corp.,2537,8465,-5928
191,University of Iowa Hospitals and Clinics Healt...,120,1370,-1250
193,The Sportsman Guide Inc.,112,1008,-896
40,McCormick and Company,2513,3360,-847
178,Ontario Securities Commission,159,980,-821
155,NATIONAL GUARDIAN LIFE INSURANCE CO,280,900,-620
261,"FOUR SEASONS, INC, A DIVISION OF ST ANDARD MOT...",20,342,-322
117,Office of Personnel Management,500,762,-262
238,ROBERT HALF INTERNATIONAL,38,150,-112


In [68]:
df.groupby(['Software_Name', 'Customer_Name']).sum().sort_values('Dropped_SS_since_Q1_2020',ascending=False).reset_index()[['Software_Name','Customer_Name', 'Dropped_SS_since_Q1_2020','Current_S&S_Quantity']].head(30)

,Software_Name,Customer_Name,Dropped_SS_since_Q1_2020,Current_S&S_Quantity
0,FileNet Content Manager Eligible Participant R...,US Department of Veterans Affairs Filenet use ...,6250000,0
1,FileNet Content Manager Employee User Value Unit,US Department of Veterans Affairs Filenet use ...,190000,0
2,WebSphere Application Server ND Processor Valu...,McDonald’s Corporation,20720,0
3,Spectrum Protect Extended Edition Processor Va...,Duke University,20456,0
4,InfoSphere DataStage Processor Value Unit,PFIZER INC,19600,0
5,InfoSphere DataStage for Non-Production Proces...,PFIZER INC,19600,0
6,Global Retention Policy and Schedule Managemen...,Citizens Bank N.A.,17953,0
7,WebSphere MQ Processor Value Unit,McDonald’s Corporation,14000,0
8,Security Identity and Access Assurance Authori...,MVP Health Plan,13258,0
9,WebSphere Application Server ND Processor Valu...,US Immigration & Customs Enforcement,11120,9000
